# Dependencies

In [1]:
from pinecone import Pinecone

from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field
from openai import OpenAI

from pydantic import BaseModel, Field, ValidationError
from typing import List, Dict, Optional


import os

from dotenv import load_dotenv
load_dotenv()

import os
import json
from crewai import Agent, Task, Crew, Process
from textwrap import dedent

from pydantic import BaseModel, Field
from typing import Optional

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
PINECONE_QA_BASE_INDEX_NAME = os.getenv("PINECONE_QA_BASE_INDEX_NAME")
PINECONE_KNOWLEDGE_BASE_INDEX_NAME = os.getenv("PINECONE_KNOWLEDGE_BASE_INDEX_NAME")

# Client Initialisation
Now, we will be using the openAI client to get embeddings, as we would be needing in our model along with neccessary intialisations.

In [2]:
openAI_client = OpenAI(api_key=OPENAI_API_KEY)
embedding_model = openAI_client.embeddings
pc = Pinecone(api_key = PINECONE_API_KEY, environment = PINECONE_API_ENV)
index = pc.Index(PINECONE_KNOWLEDGE_BASE_INDEX_NAME)

def get_embedding(text) :
    """
        Function to convert the text string into embeddings using text-embedding-3-small from OpenAI
    
        Args:
            text : A string which will contain either the text chunk or the user query
            
        Returns:
            vector : A vector of 1536 dimensions
    """
    
    try:
        response = embedding_model.create(
            input=text,
            model="text-embedding-3-small"
        )
        
        return response.data[0].embedding   
    
    except Exception as e:
        raise Exception(str(e))
    

Now, to use the above, we need to create these functions as a collective tool, which can be used by an agent to query the vector DB.

# Tool Creation
Now, we would have to create tools for this purpose, as asynchronus functions which await till the vector DB is loaded and then query the vector DB, evnetually returning the relevant texts. As the admin id will be a fixed one, we will be directly referring it from the environment.

In [3]:
@tool
def answer_query_kb(query:str):
    """
        Find relevant documents for a given query and answer the query using the relevant documents.
        
        Args:
        - query (str): The search query will be in string format which will be passed straight to the LLM to find the relevant documents.
        
        Returns:
            The relevant documents from the database, given the query
    """
    query_vector = get_embedding(query)
    results = index.query(
        vector=query_vector,
        top_k=5,
        include_values=False,
        include_metadata=True,
        filter={
            "userID": os.getenv("ADMIN_ID")
        }
    )
    
    relevant_texts = []
    for record in results['matches']:
        text = {
            'score': record['score'],
            'text': record['metadata']['chunk'],
            'name': record['metadata']['document_name'],
            'reference': int(record["metadata"]["page_number"]) + 1
        }
        relevant_texts.append(text)
    
    
    if not relevant_texts : 
        return [{
            'score' : -1.00,
            'text' : 'We could not find any relevant documents for the given query',
            'name' : "NA",
            'reference' : 0
        }]
        
    return relevant_texts
    

# Agents
Now, we will have to make the tools for the agent and make it ready to be used. We will be using three agents : 

1. Researcher
2. Writer
3. Editor

To give recommendations to any previosuly solved query before.

In [4]:
researcher = Agent(
    role = "researcher",
    goal = "To research and find the relevant answer from the existing knowledge base to the query {query}",
    backstory = """
            You work as a researcher in an edtech startup 100xDevs, where your primary job is to research and give all the 
            relevant documents/chunks from the documents uploaded buy the admin, where your task is to collate those references which will 
            aid to solve the query : {query}. Your work will be directly passed to the writer, hence make sure you provide 
            references only which you can quote with the relevant document name (you will find it as 'name') and reference page (you will find it as 'reference'). 
            You have been given a tool to fetch the most relevant documents, which will give you a list of all the relevant documents, which 
            consist of text (the actual text), name (this will be the document name), reference (this will be that text's page location) and a score for you to evaluate on which is most relevant.
            In case you get irrelevant documents, the tool has handled that too and you just need to pass it on to the writer.
    """,
    verbose = True,
    allow_delegation = False,
    tools = [answer_query_kb]
)

In [5]:
writer = Agent(
    role = "writer",
    goal = "To structure and answer the query : {query} given the relevant documents from the researcher",
    backstory = """
            You are a recommendation analyst working at an edTech startup 100xDevs, where your job is to answer the theoretical
            doubts of the students from admin uploaded documents, given the query : {query}. \n
            Your work is dependent on the work of the researcher, who will bring you all the relevant information and your job is to collate it into a proper 
            explanation of what the question was and what is it's answer according to offical documents along with it's name and reference.
            You will be writing your recommendation with acknowledging and referencing each factual statement as given by the researcher, 
            hence it is important you MUST give the document name and reference to your editor, should you find a relevant documents.\n
            In case you find the research to be irrelevant, you can just let your editor know that there is no such document or solution as per
            official documentation. There is no need to solve it yourself, only solve if your have the relevant research.
    """,
    allow_delegation = False,
    verbose = True,
)

In [6]:
editor = Agent(
    role = "editor",
    goal = "To edit the writer's recommendation to answer the query : {query} and interact with student doubts",
    backstory = """
            You are the senior editor at an edTech startup 100xDevs, where your job is to handle and answer theoretical doubts
            as and when asked by a student on our discord channel. You work is dependent on the writer's content piece, as that would have information
            on the most relevant text, document name and reference they could find to the query : {query}. In case the writer is unable to produce any results or 
            denies any relevant findings to base their content on, you do not need to pass anything to the user as handled in your task.
            In case the writer is unable to provide anything, you can just tell the student that there is no such document or solution as per
            official documentation and nudge the student to vist the official documentation on the 100xDevs website.
    """,
    allow_delegation = False,
    verbose = True,
)

# Tasks

In [7]:
class ResearcherDocuments(BaseModel) : 
    score : float = Field(...,description="The score implying the relvance of this document to the given query, with 0 being the lowest and 1 being the highest, and -1 being irrelevant")
    text : str = Field(...,description="The relevant text to the query in this document")
    name : str = Field(...,description="The name of the document from which the text is")
    reference : int = Field(...,description="The page number from which this text is refered")

In [8]:
class ResearcherOutput(BaseModel) : 
    research : List[ResearcherDocuments] = Field(...,description="The collection of relevant documents")

In [9]:
research = Task(
    description="To research and find the most relevant documents and references to the given query : {query}",
    expected_output="""A list consisting of all the relevant documents, where each document (JSON) follows the structure :
        score : The score implying the relvance of this document to the given query, with 0 being the lowest and 1 being the highest, and -1 being irrelevant
        text : The relevant text to the query in this document
        name : The name of the document from which the text is
        reference : The page number from which this text is refered
    """,
    agent=researcher,
    parameters={"query": "str"},
    output_pydantic= ResearcherOutput
)

In [10]:
class WriterOutput(BaseModel) : 
    name : Optional[str] = Field(...,description="The name of the document from which the text is refered")
    reference : Optional[List[int]] = Field(...,description="The page number(s) from which this text is refered")
    content : str = Field(...,description="The content written by the writer to answer the query, given the research")

In [11]:
write = Task(
    description="To write content providing the solution to the query : {query}, given the research done by researcher",
    expected_output="""A object consisting of all the content written by the writer, where it follows the structure : 
        name : The name of the document from which the text is refered
        reference : The page number(s) from which this text is refered. Here, each page will be an integer object in a list
        content : The content written by the writer to answer the query, given the research
    """,
    agent=writer,
    output_pydantic=WriterOutput
)

In [12]:
class EditorOutput(BaseModel) : 
    status : int = Field(...,description="The status code of the answer given by the agent")
    relevance : bool = Field(...,description="Whether the team was able to find a potentially relevant solution or not")
    solution : Optional[str] = Field(...,description="The final output given by the team, given that they are able to find a relevant solution")
    name : Optional[str] = Field(...,description="The name of the document from which the text is refered, if the solution is relevant")
    reference : Optional[List[int]] = Field(...,description="The page number(s) from which this text is refered, if the solution is relevant")

In [13]:
edit = Task(
    description="To edit and give the status of work and final output as a recommendation to the student, if any content given by writer",
    expected_output="""A object consisting of all the final output given by the editor, with the structure of it as follows : 
        status : The status code of the team's findings. If any content given by writer, then 200 else 404.
        relevance : A boolean value indicating whether the team was able to complete the task or not
        solution : The final edited solution from the editor, if none then the text "Uh-oh! We could not find any relevant documents for the given query. You can visit the official documentation on the 100xDevs website."
        name : The name of the document from which the text is refered, if the solution is relevant, else "NA"
        reference : The page number(s) from which this text is refered where each page will be an integer object in a list, if the solution is relevant, else a list with 0 as the only element
    """,
    agent=editor,
    output_pydantic=EditorOutput
)

# Crew

In [14]:
crew = Crew(
    agents=[researcher,writer,editor], 
    tasks=[research,write,edit],
    memory=True,
    verbose=True
)

# Execution

In [15]:
query = "Could you explain what does useEffect do in React?"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : Could you explain what does useEffect do in React?


# Agent: researcher
## Using tool: answer_query_kb
## Tool Input: 
"{\"query\": \"Could you explain what does useEffect do in React?\"}"
## Tool Output: 
[{'score': 0.626760185, 'text': 'The\xa0useEffect\xa0hook let s you perform side ef fects in functional component s in a \nsafe, predictable way:\nuseEffect (() => {\n  // Code here is the "effect" — this is where side effects  \nhappen\n  fetchData ();\n  // Optionally, return a cleanup function that runs when t\nhe component unmounts.\n  return () => {\n    // Cleanup code, e.g., unsubscribing from an event or c', 'name': 'W9-react', 'reference': 6}, {'score': 0.577903748, 'text': 'W9 \ue088 React\n7learing timers.\n  };\n}, [/* dependencies */ ]);\nThe f irst ar gument \xa0to\xa0useEffect\xa0is the ef fect function, wher e you put the  \ncode that per forms the side e

{'status': 200,
 'relevance': True,
 'solution': "The useEffect hook is a fundamental feature in React that allows developers to perform side effects in functional components in a safe and predictable manner. Side effects can include tasks such as data fetching, subscriptions, or manually changing the DOM. The syntax for useEffect is as follows: useEffect(() => { // Code here is the 'effect' — this is where side effects happen fetchData(); // Optionally, return a cleanup function that runs when the component unmounts. return () => { // Cleanup code, e.g., unsubscribing from an event or clearing timers. }; }, [/* dependencies */ ]); The first argument provided to useEffect is the effect function, which contains the code for the side effect. The second argument is the dependencies array, which specifies when the effect should re-run. By including specific state or props in this array, React will only execute the effect when the values change. If an empty array [] is provided, the effect 

In [16]:
query = "tell me the difference between useState and useEffect in React"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : tell me the difference between useState and useEffect in React


# Agent: researcher
## Using tool: answer_query_kb
## Tool Input: 
"{\"query\": \"tell me the difference between useState and useEffect in React\"}"
## Tool Output: 
[{'score': 0.561655641, 'text': "useSt at e\nuseState\xa0is a Hook that let s you add st ate to functional component s. It returns \nan array with the cur rent state and a function t o update it.\nimport React, { useState } from 'react';\nconst Counter = () => {", 'name': 'W9-react', 'reference': 1}, {'score': 0.539746821, 'text': "import { useState , useEffect } from 'react';\nconst useDebounce  = (value, delay) => {\n    const [debouncedValue , setDebouncedValue ] = useState\n(value);\n    useEffect (() => {\n        const handler = setTimeout (() => {\n            setDebouncedValue (value);\n        }, delay);", 'name': 'W9-react', 'reference':

{'status': 200,
 'relevance': True,
 'solution': 'In React, useState and useEffect are two fundamental Hooks that serve distinct purposes. useState is a Hook that lets you add state to functional components. It returns an array containing the current state value and a function to update it (W9-react, p. 1). In contrast, useEffect is used to handle side effects in functional components. It enables you to perform operations such as data fetching, managing subscriptions, and modifying the DOM directly, which helps improve efficiency and reliability (W9-react, p. 34). Additionally, useEffect allows for cleanup tasks, such as unsubscribing from services or clearing timers, by returning a cleanup function that React will call during component unmounting (W9-react, p. 30). Together, these Hooks enhance code organization and readability; useState is focused on state management, while useEffect takes care of effects that occur as a result of changes to state or props (W9-react, p. 7). The use o

In [17]:
query = "How do JWT tokens work?"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : How do JWT tokens work?


# Agent: researcher
## Using tool: answer_query_kb
## Tool Input: 
"{\"query\": \"How do JWT tokens work?\"}"
## Tool Output: 
[{'score': 0.233698085, 'text': 'W9 \ue088 React\n41The Cont ext API is a po werful feature in React that enables y ou to manage  \nstate across your applic ation mor e effectively, especiall y when de aling with  \ndeeply nested component s.\nThe Cont ext API provides a wa y to share values (st ate, functions, et c.) \nbetween component s without ha ving to pass props down manuall y at every \nlevel.\nJ a r g o n\nCont e x t\ue092 This is cr eated using R eact.createContext(). It serves as a cont ainer \nfor the dat a you want t o share.', 'name': 'W9-react', 'reference': 41}, {'score': 0.206192985, 'text': 'W9 \ue088 React\n30Custom hooks in R eact are a powerful feature that allo ws you to encapsulate \nand reuse statefu

{'status': 404,
 'relevance': False,
 'solution': 'Uh-oh! We could not find any relevant documents for the given query. You can visit the official documentation on the 100xDevs website.',
 'name': 'NA',
 'reference': [0]}

In [18]:
query = "yaar yeh useRef kya hota hai React mein?"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : yaar yeh useRef kya hota hai React mein?


# Agent: researcher
## Using tool: answer_query_kb
## Tool Input: 
"{\"query\": \"yaar yeh useRef kya hota hai React mein?\"}"
## Tool Output: 
[{'score': 0.514394164, 'text': "Is\xa0usePrev\xa0hook that w e have made indeed cor rect? Try out this  \nexample\xa0https://giacomocer quone.com/blog/lif e-death-usepr evious-hook/\nimport { useRef, useState , useEffect } from 'react'\nimport './App.css'\nexport const usePrev = (value) => {\n  const ref = useRef();", 'name': 'W9-react', 'reference': 35}, {'score': 0.503343284, 'text': "W9 \ue088 React\n32  useEffect (() => {\n    fetchData (); // Initial fetch\n    if (interval !== null) {\n      const fetchInterval = setInterval (() => {\n        fetchData ();\n      }, interval );\n      return () => clearInterval (fetchInterval ); // Clea\nr interval on cleanup\n    }\n  }, [url, inter

{'status': 200,
 'relevance': True,
 'solution': 'In React, useRef is a hook that provides a way to create a reference to a value or a DOM element that persists across renders but does not trigger a re-render when the value changes. The value stored in useRef persists between component re-renders, meaning the value of a ref does not get reset. useRef can be used to directly access a DOM node, such as creating a ref for an input element and calling the focus method on it. Additionally, useRef can store a mutable value that persists across renders, which is useful for keeping track of the previous state or any other value without causing re-renders. The useEffect hook can also be utilized to update the ref with the current value after each render, allowing you to return the previous value (current value of ref before it is updated).',
 'name': 'W9-react',
 'reference': [25, 26, 36]}

In [19]:
query = "bhai yaar mujhe custom Hooks ke baare mein bata na"
result = crew.kickoff(inputs={"query": query})
answer = json.loads(result.raw)
answer

# Agent: researcher
## Task: To research and find the most relevant documents and references to the given query : bhai yaar mujhe custom Hooks ke baare mein bata na


# Agent: researcher
## Using tool: answer_query_kb
## Tool Input: 
"{\"query\": \"bhai yaar mujhe custom Hooks ke baare mein bata na\"}"
## Tool Output: 
[{'score': 0.473486155, 'text': 'W9 \ue088 React\n30Custom hooks in R eact are a powerful feature that allo ws you to encapsulate \nand reuse stateful logic acr oss different component s. They are essentiall y \nJavaScript functions that c an use R eact hooks int ernally. By creating cust om \nhooks, y ou can abstract awa y comple x logic, making y our component s cleaner \nand mor e manage able.\nW h y  U s e  C u s t o m  H o o k s ?\n\ue072\ue094\x00 R eusabilit y\ue092 If you find y ourself using the same logic in mul tiple', 'name': 'W9-react', 'reference': 30}, {'score': 0.34503442, 'text': "component s, a cust om hook c an help y ou avoid code duplic ation.\n\ue07

{'status': 200,
 'relevance': True,
 'solution': 'Custom hooks in React are a powerful feature that allows you to encapsulate and reuse stateful logic across different components. They are essentially JavaScript functions that can use React hooks internally. By creating custom hooks, you can abstract away complex logic, making your components cleaner and more manageable. If you find yourself using the same logic in multiple components, a custom hook can help you avoid code duplication. Additionally, they allow you to separate business logic from UI logic, making your components more focused and easier to understand.',
 'name': 'W9-react',
 'reference': [30]}

With this, we now have an agent which will be answering the core question asked by the user, based on the Admin uploaded knowledge base. This is suppose to work as a crewAI endpoint, which can be manually invoked by the student in discord.